<a href="https://colab.research.google.com/github/liz-lewis-manchester/CNM_2025_group_09/blob/Modifications-to-Q2/Question%202%20modifications.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation

end_time = float(input("How long should the model last for, in seconds  "))
time_interval = float(input("The time interval, in seconds   "))
length = float(input("Length of model, in metres   "))
length_interval = float(input("The length interval, in metres   "))
speed = float(input("Speed of river flow, in m/s   "))
initial_concentration = str(input("Name of the .csv file   "))

def concentration_from_csv(end_time, time_interval, length, length_interval, speed, initial_concentration):
    distance = np.array([])
    for i in range(int(length / length_interval + 1)):
        distance = np.append(distance, length_interval * i)

    df = pd.read_csv(initial_concentration, encoding='latin1')

    x_data = df["Distance (m)"].values
    C_data = df["Concentration (µg/m_ )"].values

    initial_conditions = np.interp(distance, x_data, C_data, left=0.0, right=0.0)

    fig, ax = plt.subplots()

    ax.set_xlim(0, length)

    max_C0 = np.max(initial_conditions)
    ax.set_ylim(0, max_C0 * 1.1 if max_C0 > 0 else 1.0)

    ax.set_xlabel("Distance (m)")
    ax.set_ylabel("Concentration (μg/m^3)")

    A_value = 1 / time_interval + speed / length_interval
    B_value = speed / length_interval

    A_array = np.array([])
    B_array = np.array([])

    for i in range(int(length / length_interval + 1)):
        A_array = np.append(A_array, A_value)
        B_array = np.append(B_array, B_value)

    num_frames = (int(end_time / time_interval)+1)

    current_initial_conditions = initial_conditions.copy()

    all_concentrations = []

    for k in range(num_frames):
        if k == 0:
            all_concentrations.append(initial_conditions.copy())
        else:
            concentration_present = np.zeros(int(length / length_interval + 1))
            concentration_present[0] = df.iloc[0, 1]

            for i in range(1, int(length / length_interval + 1)):
                concentration_present[i] = (current_initial_conditions[i] / time_interval + B_array[i] * concentration_present[i - 1]) / A_array[i]

            current_initial_conditions = concentration_present.copy()
            all_concentrations.append(concentration_present.copy())

    plt.rcParams["animation.html"] = "jshtml"
    plt.rcParams['figure.dpi'] = 150
    plt.ioff()

    def animate(f):

        plt.cla()

        concentration = all_concentrations[f]

        plt.plot(distance, concentration)

        plt.xlim(0, length)
        plt.ylim(0, max_C0 * 1.1 if max_C0 > 0 else 1.0)
        plt.xlabel("Distance (m)")
        plt.ylabel("Concentration (μg/m³)")
        plt.title(f"Q2: Concentration at Time = {f * time_interval:.1f} s")
        plt.grid(True)

    anim = matplotlib.animation.FuncAnimation(fig, animate, frames=num_frames)
    return anim

anim = concentration_from_csv(end_time, time_interval, length, length_interval, speed, initial_concentration)
anim